# Project 5 - Time evolution on a grid

**Project deadline:** This project is due for submission on Thursday, 14.07.2022, 23:55. Please check carefully the *About the Projects* section below for further details.

**Imprtant:** You again have the choice between two projects. This one focuses on simulations on time-evolutions on grids - a very important scientific tool used in many different fields from physics, from superconductors, magnets, and frustrated glassy systems over liquid crystals and weather systems, to more exotic concepts such as spread of diseases in society and growth of cities. 

This project teaches modern analysis techniques and advanced statistical techniques, and you will need to use numpy array-operations and Fourier transforms. The simulations you will perform are based on the Monte Carlo method. 

The other project focuses on symbolic problem solving in Python.


## About the Projects
- You will get one project approximately every other week.
- Besides the homework-assignments, you need to solve the projects in order to pass the course. Your final course mark consists of the mean of your project marks. We hand-out four projects during the term and we do not consider the worst project mark for your final course mark. Projects that you do not hand in are counted with a mark of 4.
- The project needs to be submitted by uploading this notebook on eCampus. You do not need to modify the notebooks name before your upload. Your project must be on eCampus by Thursday 14th of July, 23:55. **No late uploads can be accepted!**
- **In contrast to the homework exercises, each student must hand in an own solution for the projects! Of course you can and should discuss problems with each other! However, you need to be able to explain your solution in detail to your tutor and/or the lecturers! We might ask you for an interview about your project if the solution is (close to) identical to another students submission.**

**Note:** The tutors, Nina and I are very happy to help you out with difficulties you might have with the project tasks! You can ask questions any time but please do so well in advance of the deadlines!

## This project
In this project we consider time evolution of complex systems on grids. 
First we play Game of life as a test-case, and and then we construct an Ising model and study the critical temperature for a phase transition. 

This project description is very long! Please read it all before you start working on it.

***Notes:*** 
* This is the closest we will get to real simulations in this course. Python is not very well optimized for large simulations, and would not necessarily be the first choice when compared to for instance the C-family. These simulations should not challenge jupyterhub or python too much, but you should still consider how we can optimize performance as much as possible. Parallelization of the code is an option but not a requirement. Bottom line: If you pick this project, prepare, you might have a few minutes of run-time in the end. 
* I highly recommend that you do NOT append Numpy arrays, but instead already from the start allocate space by making a large empty array, which can be filled with entries.
* We have already encountered evolution on a grid in homework from week 5, where we considered the development of a field between two capacitor-plates. In this case we did not need any boundary conditions because we had a boundary rule, but the method of implementation is very much the same!

**Periodic boundary conditions**
In this project we want to work on a grid with periodic boundary conditions. This means that a point on the grid has neighbours as shown on this figure:

<img src="figs/grid_boundaries.png"  />

The bright red are the neighbours which should be used in the Ising model. When solving the Game of Life problems you also need to consider the diagonal neighbours (pale red)


### Game of Life

Conway's Game of Life is thought to mimic population evolution. You can read about it on the [wikipedia](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) page, and this is also where you need to find the rules of time evolution and the different configurations with certain properties which you will need for testing


Your tasks:

**Task 1**
* Define a grid with variable size where your game can take place. 
* Implement the rules for living and dying on the grid
* Test your rules with a stationary pattern of your choice
* Test your rules with one of the oscillators - you can for instance use imshow to see the development for a number of timesteps. You can also create an animation as in the last project.
* If you did not do it at the first bullet point, implement periodic boundary conditions
Note that the original idea for a game of life takes place on an infinite grid. Nothing is specified for the corners in case of boundary conditions. You can either set the corner points to always zero, or copy them in a way you find suiting.
* Test your boundary conditions by programming one of the 'gliders' which walks towards the edge of the grid and come to the other side. 

**Task 2**
* Create a 50x50 grid with random starting pattern with some filling-factor by drawing a random number for each cell. If the random number is below the filling-factor, this cell should be alive.
* Simulate $\geq$ 500 time-steps on the grid for 50 different initial starting conditions. Extract the number of living cells in each time-step. Plot the number of living cells as a function of time-steps meaned over the different starting conditions.
* How does the mean number of living cells after 500 time steps depend on the filling factors? Try for filling factors 5%, 10%, 40 %, 50 % and 80 %.

**Bonus questions**
* Test how the 'survival' probability evolves as a function of grid-size. For small grid we expect to run into (this also depends slightly on how we do the corner-points) finite size effects.
* In the above text we suggest using 500 time-steps, but the dynamics are not necessarily done after 500 time steps. What do you expect to reach in steady state? You can pick a selected filling factor and simulate. Do you expect any dependency on grid-size for time required for reaching steady state?

In [54]:
# Your solution to the Game of Life here

### Ising spin model
In this exercise we explore again time-dynamic on a grid, but with a random element.

The Ising spin model describes a system of spins where each spin can take the value +1 or -1. When we program this system, a spin is equivalent to the cell we used before when programming Game of Life. 

The spin system is described by a global Hamiltonian, which measures the energy of each spin relative to it's neighbours. 
The parameter $\sigma_i$ measures the state of the $i$th spin, and the Hamiltonian is 
\begin{align}
\mathcal{H} = \frac{1}{2}J\sum_i\sum_{j\in \mathcal{N}} \sigma_i \sigma_j - H\sigma_i
\end{align}
$J$ determines the coupling between neighbouring spins. 
The sign and strength of $J$ depends on the system we describe. $J$ takes negative values for when spins want to be aligned in the same direction, and positive values where spins want to be aligned opposite of their neighbours. The the factor 1/2 corrects for the summation, which counts each point twice.
$H$ is an external field. If the spins are aligned in the same direction as the field, the energy is minimized.

Correspondingly, the energy of a single spin is given by
\begin{align}
e_i = J\sigma_i\sum_{j\in \mathcal{N}} \sigma_j - H\sigma_i
\end{align}

The Ising model can for instance describe magnetic systems, systems with glass-like behavior, and a lot of other systems, and can be used to study for instance phase transitions in these systems. The latter is possible by considering the free energy of the system relative to the energy related to the energy cost or gain of flipping a spin. This energy is governed by the temperature $T$.

In addition to the energy, the system is characterized by the magnetization. The magnetization is the sum of all spins normalized by the grid size.

The behavior of the Ising model is usually viewed in the light of classical statistical physics. I urge you to think about how the behavior you observe can be interpreted in terms of entropy and free energy. What does the critical temperature for instance refer to?


### Metropolis algorithm and implementation of the Ising model
In the following we will use the Monte Carlo principle of drawing random numbers and accepting or rejecting a change based on this random value relative to the energy of each spin. This accept/reject principle was first introduced as part of what became known as the Metropolis algorithm and was first introduced in [this paper](https://aip.scitation.org/doi/pdf/10.1063/1.1699114) which describes how the method can be used for statistical calculations on a system of interacting molecules approximated by two-dimensional spheres. 

You can read a bit more about Monte Carlo and metropolis algorithms in the [online available Monte Carlo chapter ](https://www.worldscientific.com/doi/suppl/10.1142/p877/suppl_file/p877_chap04.pdf) of Stochastic Dynamics of Complex Systems by Paolo Sibani and Henrik Jeldtoft Jensen. 

We will use the same principles to implement the Ising model.

To time evolve the Ising model, we use an algorithm which goes like this:
* Select a random spin
* Calculate energy of flipping this spin
* Discard this spin with probability
    \begin{align}
        p_{i,\mathrm{flip}} = \min\left(1,\exp\left(\frac{e_{i}^{(t-1)}-e_{i}^{(t)}}{T}\right)\right),
    \end{align}
    where $e_i^{(t)}$ is the energy of the current time-step, and $e_i^{(t-1)}$ is the energy of the previous time-step. The only difference between these time steps is the flipping of a single spin. Therefore,
    \begin{align}
    \Delta e_i^{(t)} &= e_{i}^{(t-1)}-e_{i}^{(t)} = -2 e_{i}^{(t)}.
    \end{align}
    Since we always consider the instantaneous energy cost of flipping a spin, we can omit the explicit time dependency:
    \begin{align}
    \Delta e_i &= -2 e_{i}
    \end{align}
Note the meaning of the probability-expression: If the energy is lowered (negative) by a flip, the change is always accepted. If the system energy is increased by the flip, the probability depends on the temperature. For high temperatures, $\Delta e_i/T$ goes towards 0, and the probability goes towards 1, meaning that we are more likely to accept 'bad' spin-flips.
* Select a new random spin and go again

Instead of flipping random spins one at a time, however, we will speed the time up a bit. This we do by considering two different sublattices: All even and all odd datapoints. The spins in these sublattices are fully independent, since a spin on an odd lattice site has only neighbours on even lattice sites.

### Your implementation
In the following, you will need to implement the Ising model following these steps

* Create an initial configuration: A grid with $\pm 1$ randomly distributed.

This initiation with a random configuration effectively mimics a quench - We start in a state with no long range order which is characteristic for high temperature, and then we start evolving the system with a finite temperature.

* Select a sublattice at random
* For the lattice - you can do it for the full lattice or your sublattice - calculate the cost of flipping every spin while keeping the neighbours as they are. Make sure you take the boundary conditions correctly into account.
* Update the spins in the sublattice of your configuration with flipped spins if
    \begin{align}
    - T \log(p_{i,\mathrm{flip}}) < \Delta e_i
    \end{align}
    where $p_{i,\mathrm{flip}}$ is drawn from a random, uniform distribution between 0 and 1.
* Extract the magnetization and the system energy in this configuration
* Repeat.




### Task 3
* Run your system with a gridsize of $\geq$ 40 for > 500 time steps 10 times for the same initial function for temperatures 0.1,1,2,2.5,3,5, and 100. What do you observe? What happens to the final configuration, what happens to the energy, and what happens to the magnetization as a function of time-steps? We expect to see a phase transition around 2.2-2.3. Do you see anything that indicates a phase transition? 

* What happens when you apply an external field? Try to find the minimum absolute value of H that brings your system into a fully magnetized state for T=10.

You can for instance make an animation showing the development of the system as a function of time for different temperatures or magnetic fields of your choice. If you do so, make sure to pick the starting parameters such that you get animations with as different behaviors as possible.




### Task 4 - pick time OR space
Now we return to zero field and consider correlations in the system in either space or time - pick your favorite dimension, and if you also want to try the other dimension, give it a try as a bonus exercise. 

The auto-correlation function can be calculated as a convolution between an array with itself. This procedure is very time-consuming. Luckily, we can use a Fourier transform to simplify the process, since a convolution in real space is a multiplication in inverse space.

Thus, to get the auto-correlation function of an array we need to Fourier transform it, multiply the Fourier transform by itself, and return it to real space. The squared Fourier transform is known as the power spectrum.

The correlations in space gives us an estimate for how large the domains have formed in the system. 
* Calculate the squared two-dimensional Fourier transform of the final state of your system. This is a two-dimensional power spectrum.
    You can use the build-in ```numpy.fft.fftn```-function to do the Fourier transform and take the absolute value. We do not take the real Fourier transform, but this is not of large importance. You might need to shift the resulting transform with ```numpy.fft.fftshift``` to display the transform nicely.
* Calculate the inverse Fourier transform From the two-dimensional power-spectrum. This is now the two-dimensional auto-correlation function. We expect it to be highly peaked at coordinate (0,0), corresponding to the correlation between a point in space with itself. Normalize the correlation function such that this point has height 1 at coordinate (0,0).

* Take the average over the power spectrum and the correlation function over >10 final states for three different temperatures, one far below, one close to, and one far above the critical temperature. What do you observe for the different temperatures?

Correlations in time gives us an estimate for the 'memory' of the system, and answers the question of whether a given point in time is dependent on a previous point in time. We expect the correlations in time to become longer for lower temperatures.

Time-correlation measurements work best on time-series with a constant mean. Therefore, we need to select a sensible time-window where the system is not still thermalizing. We will only work above the critical temperature.

* For each simulation run of your system, extract the energy and select an appropriate time-window (somewhere where the system seems to have realxed to a state where the energy is oscillating around some mean value
* For each simulation, calculate the power spectrum and the corresponding correlation function. 
* Average the correlation function, mean it, and plot it for three different temperatures - for instance 10, 5, and 3, approaching the critical temperature.
* Comment on what you see.

### Bonus questions
* So far we considered grids with $\geq$ 40 points. What happens to for instance the magnetization for the different temperatures when we consider grids with points between 10 and 100 - try for instance 20, 40, 60? 

* Consider how the necessary time-scale required for thermalization depends on the temperature. Thermalization refers to the point in time when no drastic changes on energy or magnetization occurs anymore. 

* When we considered the time-correlations, we only considered points above the critical temperature. What do you think happens below? To which value does the equilibrium time tend as the temperature goes to the critical temperature?

In [ ]:
# Your solution on the Ising model here

### More information and another bonus question
The Ising model features very rich dynamics, and we have only scratched the surface. 

You might have noticed that for temperatures below the critical temperature, the energy often stagnates at some fixed value, where the system is not fully magnetized, but where flips are very rare. Often, the energy then has some stair-case-behavior where for a long time nothing happens and then suddenly a the energy drops. This is an effect of the system being stuck in a local minimum and then after a lucky flip escaping to find a state of even lower energy.

Such dynamics can be very well studied by quenching the model: Starting at some fixed temperature, time-evolving until no transient dynamics are present, and then changing the temperature parameter suddenly and time-evolve the system again. As a final bonus exercise, you can try to quench your system: Start just above the critical temperature to have some larger domains developed. Allow some time evolution and then modify the temperature parameter to very low. Record the energy, and extract the size of energy steps (the difference between neighbouring energy points). How does the probability distribution of the energy differences look? Plot it with logarithmic y-axis. What happens if you apply a small magnetic field?

The Ising model refers to a one-dimensional spin-model, but this system can of course be extended to for instance the XY-model, where the spins can take any direction in the plane. Such models can for instance be used to model flow of weather systems or water. The Heisenberg-model is the next extension, which describes spins of unit length which can point in any direction in three dimensions. The model can in principle be extended to infinitely many dimensions, where other dimensionality parameters can refer to specific properties.  


